In [ ]:

import scipeds

scipeds.download_db()

from scipeds.data.completions import CompletionsQueryEngine
from scipeds.data.queries import (
    FieldTaxonomy,
    QueryFilters, 
)

engine = CompletionsQueryEngine()



Creating scipeds cache directory /Users/kevinphan981/Library/Caches/.scipeds.
Download complete.
scipeds has 1 files and is using 332MB in /Users/kevinphan981/Library/Caches/.scipeds


/Users/kevinphan981/Programming/occupational-licensing-study/.venv/lib/python3.13/site-packages/scipeds/data/queries.py:102: UserWarning: IPEDS award level coding and race and ethnicity coding changed between 2010 and 2011 datasets. For more details, see https://docs.scipeds.org/faq/#what-data-is-currently-included-in-scipeds
  warnings.warn(
/Users/kevinphan981/Programming/occupational-licensing-study/.venv/lib/python3.13/site-packages/scipeds/data/queries.py:110: UserWarning: Race/ethnicity data is not available before 1995. All race/ethnicities prior to 1995 are treated as 'unknown'.
  warnings.warn(


field_degrees_within_gender  \
ncses_field_group                     gender                                
Architecture and Environmental Design men                          365260   
                                      women                        302426   
Arts and Music                        men                         1878780   
                                      women                       2835427   
Business and Management               men                        13086119   

                                              field_degrees_total  \
ncses_field_group                     gender                        
Architecture and Environmental Design men                  667686   
                                      women                667686   
Arts and Music                        men                 4714207   
                                      women               4714207   
Business and Management               men                27946857   

                                              uni_degrees_within_gender  \
ncses_field_group                     gender                              
Architecture and Environmental Design men                      64938013   
                                      women                    89029897   
Arts and Music                        men                      64938013   
                                      women                    89029897   
Business and Management               men                      64938013   

                                              uni_degrees_total  
ncses_field_group                     gender                     
Architecture and Environmental Design men             153967910  
                                      women           153967910  
Arts and Music                        men             153967910  
                                      women           153967910  
Business and Management               men             153967910

In [6]:
completions = engine.field_totals_by_grouping(
    taxonomy=FieldTaxonomy.original_cip,
    query_filters=QueryFilters('start_year': 2010, 'end_year': 2019)
)
completions.head()

SyntaxError: invalid syntax (1195154732.py, line 3)

In [24]:
from scipeds.constants import COMPLETIONS_TABLE

df = engine.get_df_from_query(f"""
    SELECT *  
    FROM {COMPLETIONS_TABLE}
    WHERE YEAR BETWEEN 2010 AND 2019
    AND regexp_matches(CAST(CIPCODE AS VARCHAR), '^12\\.04.*$')
    ;
""")
df.head()

,year,unitid,cipcode,awlevel,majornum,cip2020,race_ethnicity,gender,n_awards,ncses_sci_group,ncses_field_group,ncses_detailed_field_group,nsf_broad_field,dhs_stem
0,2010,100760,12.0401,Award of at least 1 but less than 2 academic y...,1,12.0401,Black or African American,women,3,Non-science and engineering,Other Non-sciences or Unknown Disciplines,Other Non-sciences or Unknown Disciplines,Non-science and engineering,False
1,2010,101240,12.0401,Award of less than 1 academic year,1,12.0401,Unknown,women,1,Non-science and engineering,Other Non-sciences or Unknown Disciplines,Other Non-sciences or Unknown Disciplines,Non-science and engineering,False
2,2010,101240,12.0401,Award of less than 1 academic year,1,12.0401,Black or African American,men,1,Non-science and engineering,Other Non-sciences or Unknown Disciplines,Other Non-sciences or Unknown Disciplines,Non-science and engineering,False
3,2010,101240,12.0401,Award of less than 1 academic year,1,12.0401,Black or African American,women,2,Non-science and engineering,Other Non-sciences or Unknown Disciplines,Other Non-sciences or Unknown Disciplines,Non-science and engineering,False
4,2010,101240,12.0401,Award of less than 1 academic year,1,12.0401,Hispanic or Latino,women,1,Non-science and engineering,Other Non-sciences or Unknown Disciplines,Other Non-sciences or Unknown Disciplines,Non-science and engineering,False


In [25]:
df.shape

(146367, 14)